<a href="https://colab.research.google.com/github/rdt3784/STUDY_AI/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random

initial_values = [np.array([0,0,0]),np.array([0,0,0]),np.array([0,0,0])]
inc = [np.array([1,0,0]),np.array([0,1,0]),np.array([0,0,1])]

mat = np.array([initial_values[i%3]+inc[random.randint(0,100)%3] for i in range(30)])
print(mat)

[[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]]


In [2]:
class RNN_BASE:
  def __init__(self, input_size, hidden_size , seq_len ,lr):
    #seq_len - backward때 몇개의 과거까지 train 할지
    self.seq_len = seq_len
    self.learning_rate = lr
    self.h_size=hidden_size
    self.U = np.random.randn(hidden_size,input_size) * 0.1
    self.V = np.random.randn(hidden_size,hidden_size) * 0.1
    self.W = np.random.randn(input_size, hidden_size) * 0.1

    self.b0 = np.random.randn(hidden_size,1) * 0.1
    self.b1 = np.random.randn(input_size,1) * 0.1

    self.h0 = np.zeros((hidden_size,1))
    self.h1 = np.zeros((self.h_size, 1))
    self.X = []

  def reset(self):
    self.h0 = np.zeros((self.h_size, 1))
    self.h_state = []
    self.X = []
    self.h_state.append(self.h0)

  def forward(self, input):
    self.X.append(input)
    if len(self.X) > (self.seq_len):
      self.X.pop(0)

    self.h1 = np.tanh(self.U @ input + self.V @ self.h_state[-1] + self.b0)
    self.y = self.W @ self.h1 + self.b1
    ps = np.exp(self.y) / np.sum(np.exp(self.y))  # softmax계산

    self.h_state.append(self.h1)
    if len(self.h_state) > (self.seq_len+1):
      self.h_state.pop(0)
    return self.y

  def backward(self, target):
    db1 = (self.y-target)
    dW = db1 @ self.h_state[-1].T #(self.y-target) @ self.h1.T

    db0 = self.W.T @ db1 *(1 - self.h_state[-1]**2) # self.W.T @ (self.y-target)  *(1 - self.h1**2)
    dV = db0 @ self.h_state[-2].T # self.W.T @ (self.y-target)  *(1 - self.h1**2) @self.h0.T
    dU = db0 @ self.X[-1].T # self.W.T @ (self.y-target)  *(1 - self.h1**2) @ self.input.T
    dh = self.V.T @ db0
    for i in range(len(self.X)-1):
      db0 += dh * (1 - self.h_state[-2-i]**2)
      dV += dh * (1 - self.h_state[-2-i]**2) @ self.h_state[-3-i].T
      dU += dh * (1 - self.h_state[-2-i]**2) @ self.X[-2-i].T
      dh = self.V.T @ dh * (1 - self.h_state[-2-i]**2)


    self.U -= self.learning_rate * dU
    self.V -= self.learning_rate * dV
    self.b0 -= self.learning_rate * db0
    self.W -= self.learning_rate * dW
    self.b1 -= self.learning_rate * db1


class RNN:
  def __init__(self, input_size, hidden_size , seq_len ,lr):
    #seq_len - backward때 몇개의 과거까지 train 할지
    self.m=RNN_BASE(input_size, hidden_size , seq_len ,lr)
    self.i= input_size
  def train(self, x, y):
    self.m.reset()
    for i in range(len(x)):
      self.m.forward(x[i].reshape(self.i,1))
      self.m.backward(y[i].reshape(self.i,1))

  def predict(self, x,c):
    out=[]
    self.m.reset()
    for i in range(c):
      o=self.m.forward(x.reshape(self.i,1))
      idx=np.argmax(o)
      out.append(idx)
      x= np.zeros((self.i,1))
      x[idx]=1
    return out


In [4]:
x1=[]

for i in range(20):
  idx=np.argmax(mat[i+1])
  x1.append(idx)
print(x1)

m = RNN(3,5,3,0.03)
input =np.array([[1,0,0],[0,1,0],[0,0,1]])
target = np.array([[0,0,1],[1,0,0],[0,1,0]])
command = np.array([[1,1,1],[1,1,1],[1,1,1]]).reshape(3,3)
i= 3000
while True:
  i-=1
  if i<0:
    break

  m.train(mat[:-16],mat[1:])

print(m.predict(mat[0],14))

[1, 2, 0, 1, 1, 2, 2, 1, 1, 0, 1, 1, 2, 1, 0, 2, 1, 1, 2, 2]
[1, 2, 0, 1, 1, 2, 2, 1, 1, 0, 1, 1, 2, 1]


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

input =torch.tensor([[[1,0,0],[1,1,1]],[[0,1,0],[1,1,1]],[[0,0,1],[1,1,1]]],dtype=torch.float32)
target = torch.tensor([[0,0,1],[1,0,0],[0,1,0]],dtype=torch.float32)

command = torch.tensor([1,1,1])

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True,nonlinearity='tanh')

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


model = RNN(3, 3, 3)
criterion = nn.CrossEntropyLoss() # For regression tasks
optimizer = optim.SGD(model.parameters(), lr=0.01)


num_epochs=2000
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    outputs = model(input.reshape(3,2,3))
    loss = criterion(outputs, target)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 1000 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
    print(model(torch.tensor([[0,1,0],[1,1,1]],dtype=torch.float32).reshape(1,2,3)))
    print(model(torch.tensor([[0,0,1],[1,1,1]],dtype=torch.float32).reshape(1,2,3)))
    print(model(torch.tensor([[1,0,0],[1,1,1]],dtype=torch.float32).reshape(1,2,3)))



Epoch [1000/2000], Loss: 1.0842
Epoch [2000/2000], Loss: 0.5405
tensor([[ 1.4686, -1.1505, -0.2465]])
tensor([[-1.0597,  1.3044,  0.8558]])
tensor([[-0.5297,  0.6719,  0.6512]])
